In [1]:
## Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim  as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import transforms

In [2]:
## Create Fully connected network

class NN(nn.Module):
    
    def __init__(self,input_shape,output_shape):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_shape,50)
        self.fc2 = nn.Linear(50,output_shape)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
device

device(type='cuda')

In [4]:
# Hyperparameters
input_shape = 28*28
output_shape = 10
batch = 1024
num_epoch = 50
learning_rate = 2e-3

In [5]:
## load Dataset
train_dataset = datasets.MNIST(root = 'datasets/',train=True,download=True,transform = transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch,shuffle=True)

test_dataset = datasets.MNIST(root = 'datasets/',train=False,download=True,transform = transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset,batch_size=batch,shuffle=True)

In [6]:
## Initialize model 
model = NN(input_shape,output_shape).to(device)

In [7]:
## Losss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
data, target = next(iter(train_loader))

In [9]:
%%time
## Train Network 
for epoch in range(num_epoch):
    losses = []
    
    for batch_idx, (data,target) in enumerate(train_loader):
        # Get data to cuda 
        data = data.to(device=device)
        target =target.to(device=device)
        
        # reshape
        data = data.view(data.shape[0],-1)
        
        # forward
        scores = model(data)
        loss = criterion(scores,target)
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
        
    print(f"{epoch} --> loss {sum(losses)/len(losses)}")

0 --> loss 0.8741106820308556
1 --> loss 0.3230649995601783
2 --> loss 0.2653799514144154
3 --> loss 0.23131876476740432
4 --> loss 0.20564759409023545
5 --> loss 0.1852118436057689
6 --> loss 0.1701030988814467
7 --> loss 0.1561572253704071
8 --> loss 0.1443933475825746
9 --> loss 0.13342501349368338
10 --> loss 0.12378971695394839
11 --> loss 0.11494127812526994
12 --> loss 0.10795718040001595
13 --> loss 0.10047863007096922
14 --> loss 0.09517277385723794
15 --> loss 0.08942729859786519
16 --> loss 0.0840669334313627
17 --> loss 0.07948375701651735
18 --> loss 0.07543051678497913
19 --> loss 0.07253434988906828
20 --> loss 0.06829576876203892
21 --> loss 0.06491320529731653
22 --> loss 0.061589633509264156
23 --> loss 0.05927009627980701
24 --> loss 0.05609384084404525
25 --> loss 0.054533588557930314
26 --> loss 0.05142232276878114
27 --> loss 0.04983639003614248
28 --> loss 0.04740247509237063
29 --> loss 0.04561625875658908
30 --> loss 0.043617342204108074
31 --> loss 0.041037526

In [10]:
# check the accuracy of out trained model 
def check_accuracy(loader,model):
    for data,target in loader:
        num_correct = 0
        num_sample = 0
        model.eval()
        with torch.no_grad():
            data = data.to(device=device)
            target = target.to(device=device)

            # reshape 
            data = data.view(data.shape[0],-1)


            scores = model(data)
            _, pred = scores.max(1)
            # print(list(zip(pred,target)))
            num_correct += sum(pred == target)
            num_sample  += pred.shape[0]
    print(f'Total {num_correct} correct  / out of {num_sample} - accuracy {num_correct/num_sample :.3f} ')

  

In [11]:
# on test dataset
check_accuracy(test_loader,model)

Total 762 correct  / out of 784 - accuracy 0.972 


In [12]:
# on train datasets
check_accuracy(train_loader,model)

Total 607 correct  / out of 608 - accuracy 0.998 
